In [1]:
%matplotlib inline

In [2]:
from pylab import rcParams
rcParams['figure.figsize'] = 15,8

In [5]:
import scipy.stats
import analysis

In [6]:
data = analysis.get_full_dict()

In [7]:
x = analysis.pull_data(data, data.keys(), 'I use a wide array of commands.')
y = analysis.pull_data(data, data.keys(), 'I feel comfortable using the shell.')

In [24]:
correlations = {}

any = next(iter(data.keys()))
questions = list(data[any].keys())

for i in range(len(questions)):
    row = questions[i]
    correlations[row] = {}
    
    for j in range(i):
        col = questions[j]
        x = analysis.pull_data(data, data.keys(), row)
        y = analysis.pull_data(data, data.keys(), col)
        
        if not isinstance(x[0], (int, float)) or not isinstance(y[0], (int, float)):
            continue
        
        correlations[row][col] = scipy.stats.pearsonr(x, y)
        
correlations

{'Age': {},
 'Education': {},
 'Efficiency Rating': {},
 'Gender': {},
 'I feel comfortable using the shell.': {'I feel that I have sufficient knowledge of the shell for what I need.': (0.78332002635684184,
   0.0005516847695515607),
  'I feel that learning how to use new commands is easy.': (0.38545413978199239,
   0.15593370154971781),
  'I prefer command line tools to GUI tools.': (0.48507125007266594,
   0.066840357648254203),
  'I think I would prefer the shell if I understood it better (or already do).': (0.026718491426788248,
   0.92469708036961529),
  'I think the command line is easy to use.': (0.25286086871208668,
   0.36320888753478842),
  'I think the command line is more usable than a GUI.': (-0.15868618528048331,
   0.57216202003216166),
  'I think the shell is easy to learn.': (0.099850336658458883,
   0.72329700599963931),
  'I use a wide array of commands.': (0.54079486767046658,
   0.037382380089398759),
  'I use control structures in the shell.': (0.32871912932571296